In [5]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# Traditional Features Construction

In [10]:
def get_image():
    path = "../DataSet/Training/Normal"
    folders = os.listdir(path)
    no_of_images = []
    for folder in folders:
        files = [f for f in os.listdir(f"{path}/{folder}") if os.path.isfile(os.path.join(f"../Resized/{folder}", f))]
        no_of_images.append(len(files))
        
    collection = {}
    for folder, count in zip(folders, no_of_images):
        all_files = os.listdir(f"{path}/{folder}")
        collection[folder] = all_files
        
    return collection

In [11]:
image_collections = get_image()

In [53]:
def feature_extraction(path, folder, image):
    img = cv.imread(f"{path}/{folder}/{image}")
    print(path, folder, image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    img = cv.drawKeypoints(gray, kp, img)
    cv.imwrite(path, img)

In [55]:
def get_features():
    path = "../DataSet/Training/Feature Extracted"
    if not os.path.exists(path):
        os.mkdir(path)
    for folder, images in image_collections.items():
        for image in images:
            feature_extraction(path, folder, image)

In [56]:
get_features()

../DataSet/Training/Feature Extracted Apple Crimson Snow 0_100.jpg


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
def k_means_clustering(X):
    kmeans = KMeans(n_clusters=100, random_state=0, init="random", n_init="auto").fit(X)
    
    return kmeans

In [ ]:
def dimension_reduction(features):
    pca = PCA(n_components=2)
    transformed_data = pca.fit_transform(features)
    
    for i in range(4):
        c = np.flatnonzero(class_labels == i)
        plt.scatter(transformed_data[c, 0], transformed_data[c, 1])
    plt.show()
    
    return transformed_data

# Traditional Machine Learnign model - Support Vecotr Machine (SVM)

In [ ]:
def train_svm(kernel_type):
    parameters = [0.001, 0.1, 1.0, 10, 100]
    
    for c in parameters:
        svc = SVC(kernel=kernel_type, C=c, random_state=42)

In [ ]:
def plot_svm_performance():
    plt.plot()

In [ ]:
def train_svm_for_different_kernels():
    kernel_type = "linear", "rbf", "poly", "sigmoid"
    train_svm(kernel_type)

# Deep Learning - Training a Simple Convolution Neural Network Mode

In [ ]:
def images_classifier():
    num_nodes = 10
    input_shape = (100, 100, 1)
    
    model = keras.Sequential(
    [
        keras.Input(shape=input_shape), 
        layers.Conv2D(8, kernel_size=(3, 3), activation="relu"), 
        layers.Conv2D(8, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)), 
        layers.Flatten(), 
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(num_nodes, activation="softmax")
    ])
    
    model.summary()
    
    return model

In [ ]:
def train_model():
    batch_size = 128
    num_epochs = 30
    model = images_classifier()
    X_train, y_train, X_test, y_test = standardized_dataset()
    
    X_train = np.reshape(X_train, (X_train.shape[0], 100, 100, 1))
    X_test = np.reshape(X_test, (X_test.shape[0], 100, 100, 1))
    
    y_train = to_categorical(y_train, num_classes=4)
    y_test = to_categorical(y_test, num_classes=4)
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)
    
    return model, history, X_test, y_test

In [ ]:
def model_evaluation():
    model, history, X_test, y_test = train_model()
    score = model.evaluate(X_test, y_test, verbose=0)
    print("\n\nTest loss:", score[0])
    print("Test accuracy:", score[1])
    
    return history

In [ ]:
model = model_evaluation()